In [35]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell

import numpy as np
import reader_data

In [42]:
image_dim = 1024
input_vec_size = 12049
num_proj = 300
lstm_size = 128
learning_rate = 0.001


def init_weight(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def sent_model(sent, init_state, lstm_size):
    lstm = rnn_cell.LSTMCell(lstm_size, num_proj=num_proj, use_peepholes=True)
    outputs, _states = rnn.rnn(lstm, sent, initial_state=init_state)
    
    return outputs[-1], lstm.state_size

def embedding_feature(W, X_all_clips):
    return [tf.matmul(clip, W) for clip in X_all_clips]

def embedding_sentences(X_all_sents, init_state, sent_model):
    embed_all_sents = []
    all_states = []
    for sents in X_all_sents:
        embed_sents = []
        states = []
        for sent in sents:
            o, state = sent_model(sent, init_state, lstm_size)
            embed_sents.append(o)
            states.append(state)
        embed_all_sents.append(embed_sents)
        all_states.append(states)
    return embed_all_sents, all_states

def cost_s_func(embed_feat, embed_sent, margin):
    l = len(embed_feat)
    cost_s = tf.Variable(0)
    for i in range(l):
        for sent_o in embed_sent[i]:
            for j in range(l):
                if i != j:
                    s1 = tf.matmul(sent_o, tf.transpose(embed_feat[i], [1, 0]))
                    s2 = tf.matmul(sent_o, tf.transpose(embed_feat[j], [1, 0]))
                    d_cost_s = tf.maximum(tf.constant(0), tf.add(tf.sub(margin, s1), s2))
                    cost_s = tf.add(cost_s, d_cost_s)
    return cost_s

def cost_im_func(embed_feat, embed_sent, margin):
    l = len(embed_feat)
    cost_im = tf.Variable(0)
    for i in range(l):
        im_feat = embed_feat[i]
        for j in range(l):
            if i != j:
                for sent_o in embed_sent[j]:
                    s1 = tf.matmul(im_feat, tf.transpose(embed_sent[i][0], [1, 0]))
                    s2 = tf.matmul(im_feat, tf.transpose(sent_o, [1, 0]))
                    d_cost_im = tf.maximum(tf.constant(0), tf.add(tf.sub(margin, s1), s2))
                    cost_im = tf.add(cost_im, d_cost_im)
    return cost_im

def cost_func(W, X_all_sents, X_all_clips, margin, init_state):
    embed_feat = embedding_feature(W, X_all_clips)
    embed_sent = embedding_sentences(X_all_sents, init_state)
    cost_s = cost_s_func(embed_feat, embed_sent, margin)
    cost_im = cost_im_func(embed_feat, embed_sent, margin)
    return tf.add(cost_s, cost_im)

movie_clip_features = reader_data.movie_clip_features()
one_hot_clip_sent = reader_data.one_hot_clip_sent()


In [43]:
X_all_sents = [[[tf.placeholder("float", shape=[None, lstm_size]) for w in sent] for sent in sents] for sents in one_hot_clip_sent]
X_all_clips = [tf.placeholder("float", shape=[None, image_dim]) for clip in movie_clip_features]

W = init_weight([image_dim, num_proj])
margin = tf.constant(1.0)
init_state = tf.placeholder("float", shape=[None, 2*lstm_size])
cost = cost_func(W, X_all_sents, X_all_clips, margin, init_state)
train_op = tf.train.RMSPropOptimizer(learning_rate, 0.9).minimize(cost)

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    
    sent_list = {}
    movie_list = {}
    for X_sents, v_sents in zip(X_all_sents, one_hot_clip_sent):
        for X_sent, v_sent in zip(X_sents, v_sents):
            for X_word, v_word in zip(X_sent, v_sent):
                sent_list[X_word] = v_word
    
    for X_clip, v_clip in zip(X_all_clips, movie_clip_features):
        movie_list[X_clip] = v_clip.reshape(-1)
    feed = dict(sent_list.items() + movie_list.items())
    feed[init_state] = np.zeros((2*lstm_size, ))
    err, _ = sess.run(train_op, feed_dict=feed)
    print(err)

aaa


ValueError: Variable RNN/LSTMCell/W_0 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-31-bba8a7d81b89>", line 13, in sent_model
    outputs, _states = rnn.rnn(lstm, sent, initial_state=init_state)
  File "<ipython-input-31-bba8a7d81b89>", line 27, in embedding_sentences
    o, state = sent_model(sent, init_state, lstm_size)
  File "<ipython-input-31-bba8a7d81b89>", line 63, in cost_func
    embed_sent = embedding_sentences(X_all_sents, init_state)
